In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '013025'

library(missForest)
library(readxl)
library(tidyverse)
library(imputeLCMD)
library(factoextra)
# library(vegan)
# library(preprocessCore)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    # removing extra replicates for samples that weren't measured at all
    filter(Value == 'NA' & Replicate == 1 | Value != 'NA') %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    # removing extra replicates for samples that weren't measured at all
    filter(Value == 'NA' & Replicate == 1 | Value != 'NA') %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

In [2]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289777,Harper et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,981013,Jalava et al. 2006,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


First need to save the non-detect data.

In [3]:
nd_ws_df = ws_df %>%
    filter(Value == 'ND') %>%
    unique()

head(nd_ws_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID9020877,1-Methylnaphthalene,ND
2,WS,299223,Kasurinen et al. 2015,2,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,ND
3,WS,299223,Kasurinen et al. 2015,3,PAH,Weight,DTXSID9020877,1-Methylphenanthrene,ND
4,WS,429188,Kasurinen et al. 2018,4,PAH,Weight,DTXSID0025654,1-Methylpyrene,ND
5,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND
6,WS,821855,Erlandsson et al. 2020,2,PAH,Weight,DTXSID4020878,2-Methylnaphthalene,ND


# Chemical/Metal Background Filter

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. A variable will be removed if it has less than 50% of woodsmoke (WS) and wildfire (WF) experimental and/or non-detect data.

In [4]:
# # changing cols to numeric
# ws_df$Value = as.numeric(ws_df$Value)
# wf_df$Value = as.numeric(wf_df$Value)

# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [5]:
variable_presence_df = combined_df %>%
    # if the value isn't MAR count it as being present
    mutate(count = ifelse(Value != 'NA', 1, 0)) %>%
    group_by(Metric, Name) %>%
    # counting number of records for each variable
    mutate(group_count = n()) %>%
    # calculating the percentage of variables with data
    reframe(Variable_Presence_Percentage = (sum(count)/group_count) * 100) %>%
    arrange(-Variable_Presence_Percentage)

# viewing data that passed the filter
keep_variables_df = variable_presence_df %>%
     filter(Variable_Presence_Percentage >= 50) %>%
     unique()

head(keep_variables_df)

Metric,Name,Variable_Presence_Percentage
<chr>,<chr>,<dbl>
Volume,"1,2,4-Trimethylbenzene",100
Volume,"1,3,5-Trimethylbenzene",100
Volume,"1,3-Dihydroxynaphthalene",100
Volume,"1,4-Dichloro-2-butene, cis",100
Volume,"1,4-Dichlorobenzene",100
Volume,"1,8-Dihydroxynaphthalene",100


In [6]:
# only keeping records that passed the background filter
filtered_df = inner_join(keep_variables_df[,1:2], combined_df)

head(filtered_df)

Joining with `by = join_by(Metric, Name)`


Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,"1,2,4-Trimethylbenzene",WF,1289926,Wang et al. 2024,1,VOC,DTXSID6021402,730
Volume,"1,2,4-Trimethylbenzene",WF,1306371,Ketcherside et al. 2024,1,VOC,DTXSID6021402,130
Volume,"1,3,5-Trimethylbenzene",WF,1289926,Wang et al. 2024,1,VOC,DTXSID6026797,330
Volume,"1,3,5-Trimethylbenzene",WF,1306371,Ketcherside et al. 2024,1,VOC,DTXSID6026797,110
Volume,"1,3-Dihydroxynaphthalene",WF,1289821,Liang et al. 2021,1,PAH,DTXSID40456587,6
Volume,"1,4-Dichloro-2-butene, cis",WF,1289926,Wang et al. 2024,1,VOC,DTXSID3027405,230


In [7]:
dim(combined_df)
dim(filtered_df)

[1] 6577    9

[1] 5110    9

Initially had 6577 samples or records, 1467 were removed, leaving 5110.

# 2nd Variable Background Filter
This time variables will be retained only if woodsmoke and wildfire samples are present within volume or weight samples.

In [34]:
split_filtered_df = filtered_df %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [35]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 97

[1] 82

[1] 82

[1] 77

In [37]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 52

[1] 52

[1] 53

[1] 53

There were 97, 82, 82 and 77 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 52 variables were common between volume samples and 53 were common between weight records and will be retained.

In [40]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,1,PAH,DTXSID3023845,6187000
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,2,PAH,DTXSID3023845,18890500
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,3,PAH,DTXSID3023845,7806000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,1,PAH,DTXSID3023845,1100000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,2,PAH,DTXSID3023845,2800000
Volume,Acenaphthylene,WS,429445,Rajput 2010,1,PAH,DTXSID3023845,53500


In [45]:
vol_df %>%
    filter(Name == 'Ti')

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,Ti,WS,885041,Ihantola et al. 2020,1,Metal,NA,29.7
Volume,Ti,WS,885041,Ihantola et al. 2020,2,Metal,NA,37.9
Volume,Ti,WS,993956,Ihantola et al. 2022,1,Metal,NA,30
Volume,Ti,WS,1263484,Leese et al. 1989,1,Metal,NA,220
Volume,Ti,WS,1263480,Burnet et al. 1990,2,Metal,NA,2625
Volume,Ti,WF,1070627,Siregar et al. 2022,1,Metal,NA,NA
Volume,Ti,WF,1289909,Trieu et al. 2024,1,Metal,NA,NA
Volume,Ti,WF,1289884,Samsonova et al. 2012,1,Metal,NA,NA
Volume,Ti,WF,1289885,Samsonov et al. 2005,1,Metal,NA,NA


# QRILC Imputation

In [38]:
# # recombining data again
# second_filtered_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df, consistent_ws_weight_df, consistent_wf_weight_df)

# head(second_filtered_df) 

Metric,Name,Data,HAWC_ID,Study,Replicate,Chemical_Class,DTXSID,Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,1,PAH,DTXSID3023845,6187000
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,2,PAH,DTXSID3023845,18890500
Volume,Acenaphthylene,WS,1263480,Burnet et al. 1990,3,PAH,DTXSID3023845,7806000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,1,PAH,DTXSID3023845,1100000
Volume,Acenaphthylene,WS,1263484,Leese et al. 1989,2,PAH,DTXSID3023845,2800000
Volume,Acenaphthylene,WS,429445,Rajput 2010,1,PAH,DTXSID3023845,53500
